In [2]:
import pandas as pd
import numpy as np
import pandas_ta as pta
import ta
# from datapackage import Package
# from selenium import webdriver
# import requests
# from bs4 import BeautifulSoup
# import yfinance as yf
import os
import random
import shutil
import time
from tensorflow.keras.layers import LSTM
import tensorflow as tf
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [3]:
# Pandas_ta reference

pta_reference = {
	"ad": "Accumulation/Distribution Index ",
	"adx" : "Average Directional Movement",
	"aroon" : "Aroon Oscillator",
	"bbands" : "Bollinger bands",
	"macd" : "Moving Average Convergence Divergence",
	"obv" : "On-Balance Volume",
	"rsi" : "Relative Strength Index",
	"sma" : "Simple Moving Average",
	"stoch" : "Stochastic Oscillator"
}

In [4]:
## Indicator testing

# noc = pd.read_csv("test_data/NOC.csv")
# pta.stoch(high=noc["High"], low=noc["Low"], close=noc["Close"]).iloc[:,0]
# help(pta.stoch)
# pta.adx(high=noc["High"], low=noc["Low"], close = noc["Close"], length=30).iloc[:,0]

In [13]:
# Selects the desired number of random stocks so they can be used to perform tests
# Selected tables will be copied to the test_data folder, which is emptied beforehand

n_files = 10
sample_files = random.choices(os.listdir("data"), k=n_files)
for i in os.listdir("test_data"):
	os.remove("test_data/" + i)
for file in sample_files:
	shutil.copy2("data/" + file, "test_data/")

# Replicates the big_table creating process on a much smaller scale
# (also doesn't save it to file)

test_table = pd.DataFrame(columns=["Date", "Close", "Volume", "Code"])
for i in os.listdir("test_data"):
	little_table = pd.read_csv("test_data/" + i)
	little_table['Code'] = i.replace(".csv", "")
	test_table = pd.concat([test_table, little_table[["Date", "Close", "Volume", "Code"]]], axis=0)


In [6]:
# Replicates the big_table creating process on a much smaller scale
# (also doesn't save it to file)

start_time = time.time()
for i in os.listdir("test_sets"):
	os.remove("test_sets/" + i)
dropped_rows = 0
total_rows = 0

for i in os.listdir("test_data"):
	ticker_code = i.replace(".csv", "")
	little_table = pd.read_csv("test_data/" + i)
	little_table["Close_next_day"] = pd.Series(little_table["Close"].shift(-1))
	# bands = pta.bbands(close = little_table["Close"], length=20, std=2, mamode="ema")[["BBL_20_2.0", "BBU_20_2.0"]]
	# little_table["lower_bb"], little_table["upper_bb"] = bands.iloc[:,0], bands.iloc[:,1]
	# little_table["sma20"] = pta.sma(little_table["Close"], length=20)
	# little_table["sma50"] = pta.sma(little_table["Close"], length=50)
	# little_table["accdist"] = pta.ad(high=little_table["High"], low=little_table["Low"], close=little_table["Close"], volume=little_table["Volume"])
	# little_table["adx"] = pta.adx(high=little_table["High"], low=little_table["Low"], close = little_table["Close"], length=30).iloc[:,0]
	# little_table["aroon"] = pta.aroon(little_table["Close"], little_table["Low"], length=30).iloc[:,2]
	# little_table["macd"] = pta.apo(little_table["Close"]) #apo and macd are equivalent and this function is simpler
	# little_table["obv"] = pta.obv(close=little_table["Close"], volume=little_table["Volume"])
	# little_table["rsi"] = pta.rsi(close=little_table["Close"], length=30)
	# little_table["stoch_o"] = pta.stoch(high=little_table["High"], low=little_table["Low"], close=little_table["Close"]).iloc[:,0]
	for i in range(len(little_table)):
		little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow
		if (i % 1000 == 0):
			print(f"{i} records processed of {len(little_table)}")
	little_table["Date"] = pd.to_numeric(little_table["Date"])
	little_table["Volume"] = pd.to_numeric(little_table["Volume"])
	old_len = len(little_table)
	total_rows += old_len # counts how many rows were dropped from the original dataset
	little_table = little_table.dropna()
	dropped_rows += (old_len - len(little_table)) #counts how many rows are dropped
	features = little_table.drop(columns=["Close_next_day", ])
	labels = little_table["Close_next_day"]
	scaler = MinMaxScaler()
	features_norm = pd.DataFrame(scaler.fit_transform(features))
	X_tr, X_te, y_tr, y_te = train_test_split(features_norm, labels, train_size=0.8, shuffle=False)
	pd.DataFrame(X_tr).to_pickle("test_sets/" + ticker_code + "_X_train.pkl")
	pd.DataFrame(X_te).to_pickle("test_sets/" + ticker_code + "_X_test.pkl")
	pd.DataFrame(y_tr).to_pickle("test_sets/" + ticker_code + "_y_train.pkl")
	pd.DataFrame(y_te).to_pickle("test_sets/" + ticker_code + "_y_test.pkl")
print(f"Process executed in {time.time() - start_time}. {dropped_rows} rows of {total_rows} dropped due to NaN / None values ({round((dropped_rows / total_rows * 100), 2)}%).")

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_29184\438673665.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


0 records processed of 12551
1000 records processed of 12551
2000 records processed of 12551
3000 records processed of 12551
4000 records processed of 12551
5000 records processed of 12551
6000 records processed of 12551
7000 records processed of 12551
8000 records processed of 12551
9000 records processed of 12551
10000 records processed of 12551
11000 records processed of 12551
12000 records processed of 12551
0 records processed of 12551
1000 records processed of 12551


C:\Users\BogPoet\AppData\Local\Temp\ipykernel_29184\438673665.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


2000 records processed of 12551
3000 records processed of 12551
4000 records processed of 12551
5000 records processed of 12551
6000 records processed of 12551
7000 records processed of 12551
8000 records processed of 12551
9000 records processed of 12551
10000 records processed of 12551
11000 records processed of 12551
12000 records processed of 12551


C:\Users\BogPoet\AppData\Local\Temp\ipykernel_29184\438673665.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


...

2000 records processed of 5023
3000 records processed of 5023
4000 records processed of 5023
5000 records processed of 5023


C:\Users\BogPoet\AppData\Local\Temp\ipykernel_29184\438673665.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


0 records processed of 12731
1000 records processed of 12731
2000 records processed of 12731
3000 records processed of 12731
4000 records processed of 12731
5000 records processed of 12731
6000 records processed of 12731
7000 records processed of 12731
8000 records processed of 12731
9000 records processed of 12731
10000 records processed of 12731
11000 records processed of 12731
12000 records processed of 12731


C:\Users\BogPoet\AppData\Local\Temp\ipykernel_29184\438673665.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


0 records processed of 10312
1000 records processed of 10312
2000 records processed of 10312
3000 records processed of 10312
4000 records processed of 10312
5000 records processed of 10312
6000 records processed of 10312
7000 records processed of 10312
8000 records processed of 10312
9000 records processed of 10312
10000 records processed of 10312
Process executed in 35.69456505775452. 10 rows of 92086 dropped due to NaN / None values (0.01%).


In [229]:
i = "A.csv"
little_table = pd.read_csv("test_data/" + i)
little_table["Close_next_day"] = pd.Series(little_table["Close"].shift(-1))
# bands = pta.bbands(close = little_table["Close"], length=20, std=2, mamode="ema")[["BBL_20_2.0", "BBU_20_2.0"]]
# little_table["lower_bb"], little_table["upper_bb"] = bands.iloc[:,0], bands.iloc[:,1]
# little_table["sma20"] = pta.sma(little_table["Close"], length=20)
# little_table["sma50"] = pta.sma(little_table["Close"], length=50)
# little_table["accdist"] = pta.ad(high=little_table["High"], low=little_table["Low"], close=little_table["Close"], volume=little_table["Volume"])
# little_table["adx"] = pta.adx(high=little_table["High"], low=little_table["Low"], close = little_table["Close"], length=30).iloc[:,0]
# little_table["aroon"] = pta.aroon(little_table["Close"], little_table["Low"], length=30).iloc[:,2]
# little_table["macd"] = pta.apo(little_table["Close"]) #apo and macd are equivalent and this function is simpler
# little_table["obv"] = pta.obv(close=little_table["Close"], volume=little_table["Volume"])
# little_table["rsi"] = pta.rsi(close=little_table["Close"], length=30)
# little_table["stoch_o"] = pta.stoch(high=little_table["High"], low=little_table["Low"], close=little_table["Close"]).iloc[:,0]
for i in range(len(little_table)):
	little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow
	if (i % 1000 == 0):
		print(f"{i} records processed of {len(little_table)}")
# little_table["Date"] = pd.to_numeric(little_table["Date"])
little_table["Volume"] = pd.to_numeric(little_table["Volume"])
X_train = little_table[["Date", "Close"]]
X_test = X_train[int(len(X_train) * 0.8):]
X_train = X_train[:int(len(X_train) * 0.8)]
y_train = little_table["Close_next_day"].iloc[:len(X_train)]
y_test = little_table["Close_next_day"].iloc[len(X_train):]
x_scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_scaler = MinMaxScaler()
y_train = y_scaler.fit_transform(y_train)
y_test = y_test.transform(y_test)


C:\Users\BogPoet\AppData\Local\Temp\ipykernel_29184\302303740.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  little_table["Date"].iloc[i] = (pd.to_datetime(little_table["Date"].iloc[i][:10])).toordinal() #Should probably have breaken this down but it converts the string of Date to date and then to numeric (ordinal) for use with TensorFlow


0 records processed of 5791
1000 records processed of 5791
2000 records processed of 5791
3000 records processed of 5791
4000 records processed of 5791
5000 records processed of 5791


ValueError: Expected 2D array, got 1D array instead:
array=[24.71111298 26.92976189 24.48160744 ... 66.14961243 67.22248077
 65.69535828].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [201]:
display("X ->", len(X_test) + len(X_train) == len(little_table))
display("Y ->", len(y_test) + len(y_train) == len(little_table))

'X ->'

True

'Y ->'

True

In [186]:
y_train.iloc[len(X_train) - 1]

65.69535827636719

In [187]:
y_test.iloc[0]

65.28939819335938

In [155]:

little_table

array([[0.00000000e+00, 1.24287480e-01, 1.38803465e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.05504427e-01],
       [1.18976800e-04, 1.15135536e-01, 1.13806666e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.18458002e-01],
       [4.75907198e-04, 1.09331912e-01, 1.17377659e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.04164460e-01],
       ...,
       [9.99524093e-01, 8.17717092e-01, 8.23532293e-01, ...,
        0.00000000e+00, 0.00000000e+00, 8.08628075e-01],
       [9.99881023e-01, 8.11531615e-01, 8.18864651e-01, ...,
        0.00000000e+00, 0.00000000e+00, 8.77055262e-01],
       [1.00000000e+00, 8.48644567e-01, 8.78843616e-01, ...,
        0.00000000e+00, 0.00000000e+00,            nan]])

In [202]:
def create_windows(test_set, steps):
	X_list = []
	function_set = test_set.copy()
	for i in range(len(test_set) - steps):
		x_slice = function_set[0:steps]
		X_list.append(x_slice)
		function_set = function_set.shift(-1)
	del(function_set)
	return(np.array(X_list))

In [214]:
X_train

array([[0.00000000e+00, 2.23759734e-01],
       [1.48676777e-04, 1.99291245e-01],
       [5.94707107e-04, 2.23759734e-01],
       ...,
       [9.99702646e-01, 6.53420004e-01],
       [9.99851323e-01, 6.56297969e-01],
       [1.00000000e+00, 6.68130155e-01]])

In [223]:
steps = 3
X_list = create_windows(pd.DataFrame(X_train[:,1]), steps)

In [224]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Dropout

n_features = 1
# define model
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(steps, n_features), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, return_sequences=True))
model.add(Dropout(0.2))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
# model.add(LSTM(64, return_sequences=True))
model.add(Dense(1))
model.compile(optimizer='adam', loss='huber')
model.fit(X_list, y_train[:(steps * -1)], epochs=50, batch_size=32)

Epoch 1/50
145/145 [==============================] - 5s 5ms/step - loss: 19.0094
Epoch 2/50
145/145 [==============================] - 1s 6ms/step - loss: 10.2802
Epoch 3/50
145/145 [==============================] - 1s 6ms/step - loss: 10.0083
Epoch 4/50
145/145 [==============================] - 1s 6ms/step - loss: 9.8779
Epoch 5/50
145/145 [==============================] - 1s 5ms/step - loss: 7.4490
Epoch 6/50
145/145 [==============================] - 1s 6ms/step - loss: 4.7544
Epoch 7/50
145/145 [==============================] - 1s 5ms/step - loss: 3.7513
Epoch 8/50
145/145 [==============================] - 1s 6ms/step - loss: 3.2235
Epoch 9/50
145/145 [==============================] - 1s 6ms/step - loss: 2.7553
Epoch 10/50
145/145 [==============================] - 1s 6ms/step - loss: 2.3818
Epoch 11/50
145/145 [==============================] - 1s 5ms/step - loss: 2.0860
Epoch 12/50
145/145 [==============================] - 1s 6ms/step - loss: 1.8647
Epoch 13/50
145/145 [=

In [225]:
display(y_test.shape, X_test_list.shape)
model.evaluate(X_test_list, y_test[:(steps * -1)])

(1159,)

(1156, 3)

C:\Users\BogPoet\AppData\Local\Temp\ipykernel_29184\15072840.py:2: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  model.evaluate(X_test_list, y_test[:(steps * -1)])


37/37 [==============================] - 1s 2ms/step - loss: 30.9408


30.940793991088867

In [231]:
model.summary()

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_54 (LSTM)              (None, 3, 64)             16896     
                                                                 
 dropout_54 (Dropout)        (None, 3, 64)             0         
                                                                 
 lstm_55 (LSTM)              (None, 3, 64)             33024     
                                                                 
 dropout_55 (Dropout)        (None, 3, 64)             0         
                                                                 
 lstm_56 (LSTM)              (None, 3, 64)             33024     
                                                                 
 dropout_56 (Dropout)        (None, 3, 64)             0         
                                                                 
 dense_18 (Dense)            (None, 3, 1)            